In [1]:
import plotly.graph_objects as go
import pandas as pd
import urllib
import requests
import json

In [2]:
expend_API = requests.get('https://services6.arcgis.com/bdPqSfflsdgFRVVM/arcgis/rest/services/Proposed_Budget_Expenditures_(Fiscal_Year_2022_2023)/FeatureServer/0/query?outFields=*&where=1%3D1&f=geojson')
revenue_API = requests.get('https://services6.arcgis.com/bdPqSfflsdgFRVVM/arcgis/rest/services/FY_2022_2023_Proposed_Budget_(Revenue_Only_Reviewed_By_Budget)/FeatureServer/0/query?outFields=*&where=1%3D1&f=geojson')

In [6]:
expendtext = expend_API.text
expendjson = pd.read_json(expendtext)
expend = pd.json_normalize(expendjson.features)
expend = expend.rename(columns = {"properties.Fund_Name": "Fund_Name", 
                        "properties.Dept_Name": "Dept_Name", 
                        "properties.Account_Name":"Account_Name",
                        "properties.Budget": "Budget",
                        "properties.FID":"FID"})
filter_e = expend['Dept_Name'] != "Total All Departments"
expend = expend.where(filter_e)

In [7]:
revenuetext = revenue_API.text
revenuejson = pd.read_json(revenuetext)
revenue = pd.json_normalize(revenuejson.features)
revenue = revenue.rename(columns = {"properties.Fund_Name": "Fund_Name", 
                        "properties.Dept_Name": "Dept_Name", 
                        "properties.Account":"Account",
                        "properties.Budget": "Budget",
                        "properties.FID":"FID"})
filter = revenue['Fund_Name'] != "NET TOTAL - ALL FUNDS"
revenue = revenue.where(filter)

In [8]:
revenue.Fund_Name.unique()

array(['GENERAL FUND', 'TAX LEVY', 'AVIATION FUND', 'WATER FUND',
       'SEWER FUND', 'DOWNTOWN SPECIAL ASSESSMENT ',
       'CROUSE- MARSHALL SPECIAL ASSESSMENT ', 'LESS INTERFUND REVENUES',
       nan], dtype=object)

In [9]:
FundTotals = revenue.groupby(['Fund_Name']).sum().reset_index()
ftsource = list(range( 0 , ( len(FundTotals) ) ) )
FundTotals['source'] = ftsource
ftdict = dict(zip(FundTotals.Fund_Name,FundTotals.source))

In [10]:
ftdict

{'AVIATION FUND': 0,
 'CROUSE- MARSHALL SPECIAL ASSESSMENT ': 1,
 'DOWNTOWN SPECIAL ASSESSMENT ': 2,
 'GENERAL FUND': 3,
 'LESS INTERFUND REVENUES': 4,
 'SEWER FUND': 5,
 'TAX LEVY': 6,
 'WATER FUND': 7}

In [11]:
FundTotals

,Fund_Name,id,Budget,FID,source
0,AVIATION FUND,141.0,746413.0,141.0,0
1,CROUSE- MARSHALL SPECIAL ASSESSMENT,201.0,113750.0,201.0,1
2,DOWNTOWN SPECIAL ASSESSMENT,294.0,1024400.0,294.0,2
3,GENERAL FUND,9012.0,254299053.0,9012.0,3
4,LESS INTERFUND REVENUES,102.0,-2825000.0,102.0,4
5,SEWER FUND,285.0,7153500.0,285.0,5
6,TAX LEVY,137.0,40851161.0,137.0,6
7,WATER FUND,1815.0,27892034.0,1815.0,7


In [68]:
Dept_Totals = revenue.groupby(['Fund_Name','Dept_Name'])['Budget'].sum().reset_index()
Dept_Totals = Dept_Totals[Dept_Totals.Budget > 0]
dtsource = list(range( len( FundTotals) , 
                      ( len(Dept_Totals) + (len(FundTotals)) ) 
                     ) 
                )
Dept_Totals['Dept_Name'] = Dept_Totals['Dept_Name'] + " revenue"
Dept_Totals[ 'source' ] = dtsource
Dept_Totals[ 'target' ] =  Dept_Totals[ 'Fund_Name' ].map( ftdict )

dtdict = dict(zip(Dept_Totals.Dept_Name, Dept_Totals.source))


In [69]:
eDept_Totals = expend.groupby(['Fund_Name','Dept_Name'])['Budget'].sum().reset_index()


eDept_Totals[ 'source' ] = 3
eDept_Totals[ 'target' ] = range(( len(Dept_Totals) + (len(FundTotals)) ),
                                 ( len(Dept_Totals) + (len(FundTotals)) ) + len(eDept_Totals))

edict=dict(zip(eDept_Totals.Dept_Name,eDept_Totals.target))



In [70]:
edict

{'API': 31,
 'Assessment': 32,
 'Board of Assessment Review': 33,
 'Board of Zoning': 34,
 'Bureau of Accounts': 35,
 'Bureau of Administrative Adjudication': 36,
 'Bureau of CPC': 37,
 'Bureau of Information Technology': 38,
 'Bureau of Research': 39,
 'Citizen Review Board': 40,
 "City Clerk's Office": 41,
 'Common Council': 42,
 'DPW Main': 43,
 'Department of Audit': 44,
 'Department of Code Enforcement': 45,
 'Department of Engineering': 46,
 'Department of Law': 47,
 'Department of Neighborhood & Business': 48,
 'Division of Building Services': 49,
 'Division of Contract Compliance and Minority Affairs': 50,
 'Division of Dog Control': 51,
 'Division of Information & Service Requests': 52,
 'Division of Motor Equipment Maintenance': 53,
 'Division of Parks -Grounds Maintenance': 54,
 'Division of Purchase': 55,
 'Division of Recreation': 56,
 'Division of Sewers and Streams': 57,
 'Division of Snow and Ice Control': 58,
 'Division of Street Cleaning': 59,
 'Division of Street Rep

In [71]:
revenue[ 'target' ] = revenue[ 'Dept_Name' ].map( dtdict )

expend[ 'target' ] = expend[ 'Dept_Name' ].map( edict)

In [72]:
rdict = dict(zip(revenue.Account,revenue.FID))

rlabels = {**ftdict, **dtdict, **rdict}

rlabels

{'AVIATION FUND': 0,
 'CROUSE- MARSHALL SPECIAL ASSESSMENT ': 1,
 'DOWNTOWN SPECIAL ASSESSMENT ': 2,
 'GENERAL FUND': 3,
 'LESS INTERFUND REVENUES': 4,
 'SEWER FUND': 5,
 'TAX LEVY': 6,
 'WATER FUND': 7,
 'Aviation   revenue': 8,
 'Crouse- Marshall Special Assessment  revenue': 9,
 'Downtown Special Assessment   revenue': 10,
 'Assessment revenue': 11,
 'City Clerk revenue': 12,
 'Code Enforcement revenue': 13,
 'Federal Aid revenue': 14,
 'Finance revenue': 15,
 'Fire revenue': 16,
 'Law revenue': 17,
 'Miscellaneous Revenue revenue': 18,
 'Non-Property Tax Items revenue': 19,
 'Parks & Recreation revenue': 20,
 'Police revenue': 21,
 'Public Works revenue': 22,
 'Real Property Tax Items revenue': 23,
 'Sale of Property revenue': 24,
 'State Aid revenue': 25,
 'Surpluses & Balances revenue': 26,
 'Use of Money & Property revenue': 27,
 'Sewer  revenue': 28,
 'Tax Levy revenue': 29,
 'Water   revenue': 30,
 'Unreserved, Undesignated': 94.0,
 '410010 School District Tax Buyout': 2.0,
 '

In [73]:
revsankey = revenue[['FID','target','Budget']].rename(columns = {'FID':'source','Budget':'value'})



In [74]:
revenue.Dept_Name.unique()

array(['Surpluses & Balances', 'Real Property Tax Items',
       'Non-Property Tax Items', 'Finance', 'City Clerk',
       'Code Enforcement', 'State Aid', 'Federal Aid',
       'Miscellaneous Revenue', 'Tax Levy', 'Aviation  ', 'Water  ',
       'Sewer ', 'Downtown Special Assessment  ',
       'Crouse- Marshall Special Assessment ', 'NA', nan,
       'Parks & Recreation', 'Fire', 'Police', 'Law', 'Public Works',
       'Assessment', 'Use of Money & Property', 'Sale of Property'],
      dtype=object)

In [75]:
dtsankey = Dept_Totals[['source','target','Budget']].rename(columns = {'Budget':'value'})

esankey = eDept_Totals[['source','target','Budget']].rename(columns = {'Budget':'value'})

r_e_combined = pd.concat([dtsankey,esankey])

revsankey = pd.concat([dtsankey , revsankey])

In [76]:
colordict = {0:"rgba(27,158,119,0.5)",
             1:"rgba(217,95,2,0.5)",
             2:"rgba(117,112,179,0.5)",
             3:"rgba(231,41,138,0.5)",
             4:"rgba(102,166,30,0.5)",
             5:"rgba(230,171,2,0.5)",
             6:"rgba(166,118,29,0.5)",
             7:"rgba(27,158,119,0.5)",
             8:"rgba(255,127,0,0.5)"
            }

In [77]:
nodecolors = pd.concat([FundTotals.source.map(colordict), dtsankey.target.map(colordict)])

linkcolors = dtsankey.target.map(colordict)

In [79]:
#revsankey

rsdict = dict( source = revsankey['source'].to_list(),
                target = revsankey['target'].to_list(),
                value = revsankey['value'].to_list() )

dept_sankey = dict( source = dtsankey[ 'source' ].to_list(),
                           target = dtsankey[ 'target' ].to_list(),
                           value = dtsankey[ 'value' ].to_list() ,
                           color = linkcolors)

r_e_sankey = dict( source = r_e_combined[ 'source' ].to_list(),
                   target = r_e_combined[ 'target' ].to_list(),
                   value = r_e_combined[ 'value' ].to_list() )


In [80]:
rllist = list(rlabels.keys())

dlabels = {**ftdict, **dtdict, **edict}

dllist = list(dlabels.keys())

In [81]:
dtdict

{'Aviation   revenue': 8,
 'Crouse- Marshall Special Assessment  revenue': 9,
 'Downtown Special Assessment   revenue': 10,
 'Assessment revenue': 11,
 'City Clerk revenue': 12,
 'Code Enforcement revenue': 13,
 'Federal Aid revenue': 14,
 'Finance revenue': 15,
 'Fire revenue': 16,
 'Law revenue': 17,
 'Miscellaneous Revenue revenue': 18,
 'Non-Property Tax Items revenue': 19,
 'Parks & Recreation revenue': 20,
 'Police revenue': 21,
 'Public Works revenue': 22,
 'Real Property Tax Items revenue': 23,
 'Sale of Property revenue': 24,
 'State Aid revenue': 25,
 'Surpluses & Balances revenue': 26,
 'Use of Money & Property revenue': 27,
 'Sewer  revenue': 28,
 'Tax Levy revenue': 29,
 'Water   revenue': 30}

TOO COMPLICATED!!! Why not do one by department? 

In [85]:
fig = go.Figure(data = [go.Sankey(
    node = dict(
        pad = 15,
        thickness = 20,
        line = dict(color = "black", width = 0.5),
        label = dllist,
        #color = nodecolors
    ),
    link = r_e_sankey
)])

fig.update_layout(title_text="City of Syracuse Budget Sankey Diagram, projected 2022-2023", font_size=10, width =1000, height = 1500, margin_b = 150)
fig.show()